# Vulkan

In this section we derive the projection matrices for Vulkan. The basic process is the following.

1. Define the view space orthonormal frame.
2. Define the clip space orthonormal frame compatible with the target canonical view volume. The target clip space should have the same orthonormal frame as the target canonical view volume.
3. Define the target canonical view volume's view volume.
4. Define the frustum parameters for the view space viewing frustum.
5. Using the frustum parameters and the canonical view volume parameters, construct a canonical version of the projection of interest.
6. Construct a coordinate transformation from the view space orthonormal frame to the canonincal view space orthonormal frame.
7. Construct a coordinate transformation from the canonical clip space to the target clip space.
8. Multiply the resulting matrices together in the right order to get the final result.

We now proceed to do this for Vulkan's left-handed and right-handed view space coordinate frames.

In [16]:
import sys
import os

sys.path.insert(0, os.path.abspath('../projection_matrices'))

In [17]:
import sympy
import projection_matrices as pm

In [18]:
def equation_for_display(lhs, rhs) -> sympy.Eq:
    expr_lhs = sympy.Symbol('expr_lhs')
    expr_rhs = sympy.Symbol('expr_rhs')
    expr_lhs = sympy.UnevaluatedExpr(lhs)
    expr_rhs = sympy.UnevaluatedExpr(rhs)

    return sympy.Eq(expr_lhs, expr_rhs)

In [19]:
def change_of_orientation_lh_to_rh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0, -1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_rh_to_lh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0, -1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_lh_to_lh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0,  1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_rh_to_rh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0,  1, 0],
        [0, 0,  0, 1]
    ])

In [20]:
def rotation_x(angle: sympy.Symbol) -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,                 0,                0],
        [0, sympy.cos(angle), -sympy.sin(angle), 0],
        [0, sympy.sin(angle),  sympy.cos(angle), 0],
        [0, 0,                 0,                1]
    ])

## Vector Space Details For The Vulkan Left-Handed Orthornormal Frames

The **view space** is a vector space $V_{\text{view}} := (\mathbb{R}^3, O_{\text{view}}, B_{\text{view}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{view}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{view}} := \{ \hat{x}, -\hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $-\hat{z}$ points towards the viewer out of the view volume.
* The orthonormal frame $(O_{\text{view}}, B_{\text{view}})$ has a left-handed orientation.

The **clip space** is a vector space $V_{\text{clip}} := (\mathbb{R}^3, O_{\text{clip}}, B_{\text{clip}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{clip}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{clip}} := \{ \hat{x}, -\hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{clip}}, B_{\text{clip}})$ has a right-handed orientation.

The **canonical view volume** is the vector space $V_{\text{cvv}} := (\mathbb{R}^3, O_{\text{cvv}}, B_{\text{cvv}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{cvv}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{cvv}} := \{ \hat{x}, -\hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{cvv}}, B_{\text{cvv}})$ has a right-handed orientation.
* The **view volume** is parametrized by $[-1, 1] \times [-1, 1] \times [0, 1]$.

## Derivation Of The Projection Matrices For The Left-Handed View Space Orthonormal Frame

In each case, we map the Vulkan view space coordinate system to canonical view space coordinate system, apply the canonical transformation, then apply the clip space coordinate system transformation to get to Vulkan's clip space. In the left-handed case, we map Vulkan's view space coordinate frame to the canonical view space coordinate frame with a rotation of $\pi$ radians about the x-axis. We map the canonical clip space coordinate frame to Vulkan's clip space frame using a rotation of $\pi$ radians about the x-axis and a change of orientation flipping the z-axis. This results in the desired projection matrices.

In [21]:
def perspective_lh_vulkan() -> sympy.Matrix:
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds_vk = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_can_lh_lh = pm.perspective(frustum_bounds_vk, ndc_bounds_vk)
    
    x_lh_lh = change_of_orientation_lh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()

    m_per_lh_rh = (x_lh_rh * c_vk_inv) * m_per_can_lh_lh * (c_vk * x_lh_lh)

    return m_per_lh_rh

def perspective_fov_lh_vulkan() -> sympy.Matrix:
    aspect, theta_vfov, n, f = sympy.symbols('aspect theta_vfov n f')
    frustum_fov_bounds_vk = pm.FrustumFovBounds(aspect, theta_vfov, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_fov_can_lh_lh = pm.perspective_fov(frustum_fov_bounds_vk, ndc_bounds_vk)

    x_lh_lh = change_of_orientation_lh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()

    m_per_fov_lh_rh = (x_lh_rh * c_vk_inv) * m_per_fov_can_lh_lh * (c_vk * x_lh_lh)

    return m_per_fov_lh_rh

def orthographic_lh_vulkan() -> sympy.Matrix:
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds_vk = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_orth_can_lh_lh = pm.orthographic(frustum_bounds_vk, ndc_bounds_vk)

    x_lh_lh = change_of_orientation_lh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()

    m_orth_lh_rh = (x_lh_rh * c_vk_inv) * m_orth_can_lh_lh * (c_vk * x_lh_lh)

    return m_orth_lh_rh

The left-handed frustum perspective projection matrix is given by

In [22]:
m_per_vk_sym = sympy.Symbol(r'M^{Vulkan}_{per, lh}')
m_per_vk = perspective_lh_vulkan()
equation_for_display(m_per_vk_sym, m_per_vk)

Eq(M^{Vulkan}_{per, lh}, Matrix([
[2*n/(l + r),           0, -(l - r)/(l + r),            0],
[          0, 2*n/(b + t),  (b - t)/(b + t),            0],
[          0,           0,       -f/(f - n), -f*n/(f - n)],
[          0,           0,               -1,            0]]))

The left-handed symmetric vertical field of view perspective projection matrix is given by

In [23]:
m_per_fov_vk_sym = sympy.Symbol(r'M^{Vulkan}_{per, fov, lh}')
m_per_fov_vk = perspective_fov_lh_vulkan()
equation_for_display(m_per_fov_vk_sym, m_per_fov_vk)

Eq(M^{Vulkan}_{per, fov, lh}, Matrix([
[1/(aspect*tan(theta_vfov/2)),                   0,          0,            0],
[                           0, 1/tan(theta_vfov/2),          0,            0],
[                           0,                   0, -f/(f - n), -f*n/(f - n)],
[                           0,                   0,         -1,            0]]))

The left-handed frustum orthographic projection matrix is given by

In [24]:
m_orth_vk_sym = sympy.Symbol(r'M^{Vulkan}_{orth, lh}')
m_orth_vk = orthographic_lh_vulkan()
equation_for_display(m_orth_vk_sym, m_orth_vk)

Eq(M^{Vulkan}_{orth, lh}, Matrix([
[2/(l + r),         0,          0,  (l - r)/(l + r)],
[        0, 2/(b + t),          0, -(b - t)/(b + t)],
[        0,         0, -1/(f - n),       -n/(f - n)],
[        0,         0,          0,                1]]))

## Vector Space Details For The Vulkan Right-Handed Orthornormal Frames

The **view space** is a vector space $V_{\text{view}} := (\mathbb{R}^3, O_{\text{view}}, B_{\text{view}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{view}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{view}} := \{ \hat{x}, -\hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{view}}, B_{\text{view}})$ has a right-handed orientation.

The **clip space** is a vector space $V_{\text{clip}} := (\mathbb{R}^3, O_{\text{clip}}, B_{\text{clip}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{clip}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{clip}} := \{ \hat{x}, -\hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{clip}}, B_{\text{clip}})$ has a right-handed orientation.

The **canonical view volume** is the vector space $V_{\text{cvv}} := (\mathbb{R}^3, O_{\text{cvv}}, B_{\text{cvv}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{cvv}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{cvv}} := \{ \hat{x}, -\hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $-\hat{y}$ points down, and the basis vector $\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{cvv}}, B_{\text{cvv}})$ has a right-handed orientation.
* The **view volume** is parametrized by $[-1, 1] \times [-1, 1] \times [0, 1]$.

In particular, each vector space has the same orthonormal frame.

## Derivation Of The Projection Matrices For The Right-Handed View Space Orthonormal Frame

In each case, we map the Vulkan view space coordinate system to canonical view space coordinate system, apply the canonical transformation, then apply the clip space coordinate system transformation to get to Vulkan's clip space. In the right-handed case, Vulkan's view space coordinate frame and clip space coordinate frame are identical, so the clip space transformation is the inverse of the view space one. This results in the desired projection matrices.

In [25]:
def perspective_rh_vulkan() -> sympy.Matrix:
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds_vk = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_can_lh_lh = pm.perspective(frustum_bounds_vk, ndc_bounds_vk)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()
    
    m_per_rh_rh = (x_lh_rh * c_vk_inv) * m_per_can_lh_lh * (c_vk * x_rh_lh)

    return m_per_rh_rh

def perspective_fov_rh_vulkan() -> sympy.Matrix:
    aspect, theta_vfov, n ,f = sympy.symbols('aspect theta_vfov n f')
    frustum_fov_bounds_vk = pm.FrustumFovBounds(aspect, theta_vfov, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_fov_can_lh_lh = pm.perspective_fov(frustum_fov_bounds_vk, ndc_bounds_vk)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()
    
    m_per_fov_rh_rh = (x_lh_rh * c_vk_inv) * m_per_fov_can_lh_lh * (c_vk * x_rh_lh)

    return m_per_fov_rh_rh

def orthographic_rh_vulkan() -> sympy.Matrix:
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds_vk = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds_vk = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_orth_can_lh_lh = pm.orthographic(frustum_bounds_vk, ndc_bounds_vk)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_rh = change_of_orientation_lh_to_rh()
    c_vk = rotation_x(sympy.pi)
    c_vk_inv = c_vk.inv()
    
    m_orth_rh_rh = (x_lh_rh * c_vk_inv) * m_orth_can_lh_lh * (c_vk * x_rh_lh)

    return m_orth_rh_rh

The right-handed frustum perspective projection matrix is given by

In [26]:
m_per_vk_sym = sympy.Symbol(r'M^{Vulkan}_{per, rh}')
m_per_vk = perspective_rh_vulkan()
equation_for_display(m_per_vk_sym, m_per_vk)

Eq(M^{Vulkan}_{per, rh}, Matrix([
[2*n/(l + r),           0,  (l - r)/(l + r),            0],
[          0, 2*n/(b + t), -(b - t)/(b + t),            0],
[          0,           0,        f/(f - n), -f*n/(f - n)],
[          0,           0,                1,            0]]))

The right-handed symmetric vertical field of view perspective projection matrix is given by

In [27]:
m_per_fov_vk_sym = sympy.Symbol(r'M^{Vulkan}_{per, fov, rh}')
m_per_fov_vk = perspective_fov_rh_vulkan()
equation_for_display(m_per_fov_vk_sym, m_per_fov_vk)

Eq(M^{Vulkan}_{per, fov, rh}, Matrix([
[1/(aspect*tan(theta_vfov/2)),                   0,         0,            0],
[                           0, 1/tan(theta_vfov/2),         0,            0],
[                           0,                   0, f/(f - n), -f*n/(f - n)],
[                           0,                   0,         1,            0]]))

The right-handed frustum orthographic projection matrix is given by

In [28]:
m_orth_vk_sym = sympy.Symbol(r'M^{Vulkan}_{orth, rh}')
m_orth_vk = orthographic_rh_vulkan()
equation_for_display(m_orth_vk_sym, m_orth_vk)

Eq(M^{Vulkan}_{orth, rh}, Matrix([
[2/(l + r),         0,         0,  (l - r)/(l + r)],
[        0, 2/(b + t),         0, -(b - t)/(b + t)],
[        0,         0, 1/(f - n),       -n/(f - n)],
[        0,         0,         0,                1]]))